In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

In [55]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("display.max_columns", None)

sns.set_theme(palette="pastel")
plt.rcParams["figure.figsize"] = (12, 6)

COLUMNAS_GRD = [
    "RUT",
    "SEXO",
    # "EDAD_CANT",
    "PREVI",
    "BENEF",
    "NOM_HOPI_P",
    "FECHA_INGRESO",
    "FECHA_EGRESO",
    "ANO_EGR",
    "DIAS_ESTAD",
    "DIAG1",
    "DIAG2",
    "DIAG3",
    "INTERV_Q",
    "COD_FONASA",
    "INTERV_Q_PPAL",
    "INTERV_Q_2",
    "INTERV_Q_3",
    "T_DIAG1",
    "T_DIAG2",
    "T_DIAG3",
    "PROCED",
    "PROCED_PPAL",
    "PROCED_2",
    "PROCED_3",
]

In [72]:
# Lee GRD del INT
df_egresos = pd.read_csv("../data/processed/df_procesada.csv", usecols=COLUMNAS_GRD)
# Formatea fechas de GRD
df_egresos["FECHA_INGRESO"] = pd.to_datetime(df_egresos["FECHA_INGRESO"])
df_egresos["FECHA_EGRESO"] = pd.to_datetime(df_egresos["FECHA_EGRESO"])

# Lee la base de pabellon del INT
df_pabellon = pd.read_csv(
    "../data/external/datos_limpios_pabellon.csv", encoding="latin-1", sep=";"
)
# Formatea fechas de Pabellon
df_pabellon["fecha"] = pd.to_datetime(df_pabellon["fecha"])

# Formatea duraciones de operaciones
df_pabellon["duracion"] = pd.to_timedelta(df_pabellon["duracion"])

/var/folders/wy/2slj_t7s2vz19v2h2926qb340000gn/T/ipykernel_56595/4143286530.py:2: DtypeWarning: Columns (80,156) have mixed types. Specify dtype option on import or set low_memory=False.
  df_egresos = pd.read_csv("../data/processed/df_procesada.csv", usecols=COLUMNAS_GRD)


In [73]:
# Une la base de GRD y Pabellon segun el RUT del Paciente
grd_y_pabellon = df_egresos.merge(df_pabellon, how="inner", left_on="RUT", right_on="ID_PACIENTE")

# Filtra la union de GRD y Pabellon para dejar las Int. Q que se hicieron dentro del egreso
grd_y_pabellon = grd_y_pabellon.query("fecha >= FECHA_INGRESO and fecha <= FECHA_EGRESO")

In [80]:
resumen_duraciones = (
    grd_y_pabellon.groupby(["ANO_EGR", "DIAG1"])["duracion"].describe()
    .reset_index()
)

In [85]:
resumen_duraciones.query("DIAG1 == 'C341'")

,ANO_EGR,DIAG1,count,mean,std,min,25%,50%,75%,max
15,2015,C341,86,0 days 02:41:58.604651162,0 days 01:49:14.809848328,0 days 00:20:00,0 days 01:21:15,0 days 02:05:00,0 days 03:40:00,0 days 09:00:00
254,2016,C341,70,0 days 03:10:21.428571428,0 days 01:47:23.261683185,0 days 00:25:00,0 days 01:55:00,0 days 02:40:00,0 days 04:48:45,0 days 07:30:00
491,2017,C341,77,0 days 03:24:24.935064935,0 days 02:14:04.543201740,0 days 00:20:00,0 days 01:35:00,0 days 02:45:00,0 days 05:10:00,0 days 08:45:00
702,2018,C341,104,0 days 03:29:42.692307692,0 days 01:44:08.237690301,0 days 00:35:00,0 days 02:00:00,0 days 02:57:30,0 days 04:45:00,0 days 07:55:00
924,2019,C341,107,0 days 02:57:06.168224299,0 days 01:50:38.997657475,0 days 00:05:00,0 days 01:37:30,0 days 02:30:00,0 days 04:00:00,0 days 08:50:00
1147,2020,C341,46,0 days 03:16:18.260869565,0 days 01:34:38.885377469,0 days 00:40:00,0 days 02:06:15,0 days 03:02:30,0 days 04:00:00,0 days 07:35:00
1322,2021,C341,57,0 days 03:11:24.210526315,0 days 01:37:13.701026721,0 days 00:35:00,0 days 01:55:00,0 days 02:55:00,0 days 04:05:00,0 days 07:00:00
1544,2022,C341,89,0 days 02:34:43.146067415,0 days 01:29:03.331356914,0 days 00:20:00,0 days 01:20:00,0 days 02:00:00,0 days 03:45:00,0 days 05:55:00
